In [1]:
import pandas as pd
from copy import deepcopy
import os
from joblib import Parallel, delayed

import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from tqdm import tqdm
from glob import glob

import pwlf

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def canbefloat(x):
    try:
        float(x)
    except:
        return False
    else:
        return True
ucanbefloat = lambda t: canbefloat(t)
vfunc = np.vectorize(ucanbefloat)

In [ ]:
files = os.listdir('/scratch/ab9738/traffic/data/Sao_Paulo/per-segment/')[:5]
jams_df = pd.DataFrame(columns=['osm_way_id', 'utc_timestamp', 'speed_kph_mean'])
cols = pd.read_csv('/scratch/ab9738/traffic/data/Sao_Paulo/movement-speeds-hourly-sao-paulo-2018-1.csv',nrows=1).columns
# data_df = pd.DataFrame(columns=cols)
for test_file in files:
    try:
        wayid = test_file.split(".")[0]
        test_file = '/scratch/ab9738/traffic/data/Sao_Paulo/per-segment/'+test_file

        df = pd.read_csv(test_file,error_bad_lines=False,header=None,warn_bad_lines=False)
        df.columns = cols
        df = df.dropna()
        timestamp = df['utc_timestamp']
        timestamp = pd.to_datetime(timestamp, format="%Y-%m-%dT%H:%M:%S.000Z", errors='coerce')
        df['utc_timestamp'] = timestamp
        df = df.dropna()
        df = df.sort_values(by='utc_timestamp')
        tmp = df['speed_kph_mean'].to_numpy()
        df = df[vfunc(tmp)]

        series = tmp[vfunc(tmp)].astype(np.float)
        indices = np.cumsum(np.random.poisson(8, int(len(series)/8)))
        indices = indices[indices<len(series)]
        speed_sample = series[indices]

        speed_sample.sort()
        speed_sample = speed_sample[::-1]

        ind = np.arange(len(speed_sample))
        ind = ind/len(speed_sample)

        pwlf_func = pwlf.PiecewiseLinFit(ind, speed_sample)
        breaks = pwlf_func.fit(3, x_c=[ind[0],ind[-1]], y_c=[speed_sample[0],speed_sample[-1]])
        x_lin = np.linspace(ind[0], ind[-1], 1000)
        y_lin = pwlf_func.predict(x_lin)
        s_vals = pwlf_func.predict(breaks)
        s1, s2 = s_vals[2], s_vals[1]
        jam_th = (s1+s2)/4
        jams_df_ = df[series<jam_th]
        jams_df_ = jams_df_[['osm_way_id', 'utc_timestamp', 'speed_kph_mean']]

        if(len(jams_df_)):
            jams_df_ = jams_df_.reset_index()
            df_copy_ = deepcopy(jams_df_)
            i = 1
            while(i != len(jams_df_)):
                if(jams_df_.iloc[-i]['utc_timestamp']-pd.Timedelta(hours=1)==jams_df_.iloc[-i-1]['utc_timestamp']):
                    df_copy_ = df_copy_.drop(len(jams_df_)-i)
                i = i+1
            jams_df_ = df_copy_
        jams_df = pd.concat([jams_df, jams_df_])
        # data_df = pd.concat([data_df, df])

    except:
        continue

In [ ]:
jams_df = jams_df.reset_index(drop=True)

In [ ]:
# data_df = data_df.reset_index(drop=True)